In [1]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
sqlContext = SQLContext(sc)

ratings_rdd = sc.textFile('/Users/jitu/Documents/Fall/Big Data/Assignments/ml-latest-small/ratings.csv').map(lambda line: line.split(",")[:-1])

df = ratings_rdd.toDF(['user_ID','movie_ID','ratings'])


In [2]:
df.show()

+-------+--------+-------+
|user_ID|movie_ID|ratings|
+-------+--------+-------+
| userId| movieId| rating|
|      1|      31|    2.5|
|      1|    1029|    3.0|
|      1|    1061|    3.0|
|      1|    1129|    2.0|
|      1|    1172|    4.0|
|      1|    1263|    2.0|
|      1|    1287|    2.0|
|      1|    1293|    2.0|
|      1|    1339|    3.5|
|      1|    1343|    2.0|
|      1|    1371|    2.5|
|      1|    1405|    1.0|
|      1|    1953|    4.0|
|      1|    2105|    4.0|
|      1|    2150|    3.0|
|      1|    2193|    2.0|
|      1|    2294|    2.0|
|      1|    2455|    2.5|
|      1|    2968|    1.0|
+-------+--------+-------+
only showing top 20 rows



In [3]:
given_userID = 1
given_movieID = '31'
avgrating = '3.0'

In [4]:
movies_rdd = sc.textFile('/Users/jitu/Documents/Fall/Big Data/Assignments/ml-latest-small/movies.csv').map(lambda line: line.split(",")[:2])

df1 = movies_rdd.toDF(['movie_ID','movie_name'])


In [5]:
df1.show()

+--------+--------------------+
|movie_ID|          movie_name|
+--------+--------------------+
| movieId|               title|
|       1|    Toy Story (1995)|
|       2|      Jumanji (1995)|
|       3|Grumpier Old Men ...|
|       4|Waiting to Exhale...|
|       5|Father of the Bri...|
|       6|         Heat (1995)|
|       7|      Sabrina (1995)|
|       8| Tom and Huck (1995)|
|       9| Sudden Death (1995)|
|      10|    GoldenEye (1995)|
|      11| "American President|
|      12|Dracula: Dead and...|
|      13|        Balto (1995)|
|      14|        Nixon (1995)|
|      15|Cutthroat Island ...|
|      16|       Casino (1995)|
|      17|Sense and Sensibi...|
|      18|   Four Rooms (1995)|
|      19|Ace Ventura: When...|
+--------+--------------------+
only showing top 20 rows



In [6]:
df.createOrReplaceTempView("movie_ratings")

In [7]:
df1.createOrReplaceTempView("all_movies")
movies = spark.sql("SELECT * FROM all_movies where movie_ID>0")
movies.show()

+--------+--------------------+
|movie_ID|          movie_name|
+--------+--------------------+
|       1|    Toy Story (1995)|
|       2|      Jumanji (1995)|
|       3|Grumpier Old Men ...|
|       4|Waiting to Exhale...|
|       5|Father of the Bri...|
|       6|         Heat (1995)|
|       7|      Sabrina (1995)|
|       8| Tom and Huck (1995)|
|       9| Sudden Death (1995)|
|      10|    GoldenEye (1995)|
|      11| "American President|
|      12|Dracula: Dead and...|
|      13|        Balto (1995)|
|      14|        Nixon (1995)|
|      15|Cutthroat Island ...|
|      16|       Casino (1995)|
|      17|Sense and Sensibi...|
|      18|   Four Rooms (1995)|
|      19|Ace Ventura: When...|
|      20|  Money Train (1995)|
+--------+--------------------+
only showing top 20 rows



In [8]:
ratings = spark.sql("SELECT * FROM movie_ratings where movie_ID>0")
ratings.show()

+-------+--------+-------+
|user_ID|movie_ID|ratings|
+-------+--------+-------+
|      1|      31|    2.5|
|      1|    1029|    3.0|
|      1|    1061|    3.0|
|      1|    1129|    2.0|
|      1|    1172|    4.0|
|      1|    1263|    2.0|
|      1|    1287|    2.0|
|      1|    1293|    2.0|
|      1|    1339|    3.5|
|      1|    1343|    2.0|
|      1|    1371|    2.5|
|      1|    1405|    1.0|
|      1|    1953|    4.0|
|      1|    2105|    4.0|
|      1|    2150|    3.0|
|      1|    2193|    2.0|
|      1|    2294|    2.0|
|      1|    2455|    2.5|
|      1|    2968|    1.0|
|      1|    3671|    3.0|
+-------+--------+-------+
only showing top 20 rows



In [9]:
sql1 = spark.sql("SELECT count(distinct(movie_ratings.user_ID)) FROM movie_ratings where movie_ratings.user_ID in (SELECT user_ID FROM movie_ratings where movie_ratings.movie_ID = '%s')" %given_movieID)
sql1.show()

+-----------------------+
|count(DISTINCT user_ID)|
+-----------------------+
|                     42|
+-----------------------+



In [10]:
avg_ratings = spark.sql("select movie_ID, avg(ratings) as avgrating from movie_ratings group by movie_ID having avg(ratings)>3.0")
avg_ratings.show()

+--------+------------------+
|movie_ID|         avgrating|
+--------+------------------+
|    2294|3.2735849056603774|
|     296| 4.256172839506172|
|    3210|3.6153846153846154|
|   88140| 3.659090909090909|
|  115713|3.9423076923076925|
|   27317|              3.75|
|    1090|3.9338235294117645|
|    3959|            3.4375|
|    3606| 4.111111111111111|
|    6731|              3.25|
|   89864|3.6923076923076925|
|  106022|               4.0|
|   48738| 4.033333333333333|
|    2069|3.7777777777777777|
|    3414|               4.0|
|    4821| 4.333333333333333|
|    1572| 4.166666666666667|
|     691| 3.357142857142857|
|   50802|              3.25|
|   26082|               4.5|
+--------+------------------+
only showing top 20 rows



In [11]:
users_movies = spark.sql("select movie_ID, avg(ratings) as AVG, count(movie_ID) as count from movie_ratings where movie_ID in (SELECT distinct(movie_ID) FROM movie_ratings where user_ID in (SELECT user_ID FROM movie_ratings where movie_ID = '%s' and user_ID not in ('%s'))) group by movie_ID having count >=5 order by AVG desc" %(given_movieID, given_userID))
users_movies.show()

+--------+-----------------+-----+
|movie_ID|              AVG|count|
+--------+-----------------+-----+
|     116|             4.75|    6|
|    7116|              4.7|    5|
|     178|              4.7|    5|
|    1939|4.636363636363637|   11|
|    5114|              4.6|    5|
|    2924|4.583333333333333|    6|
|    3469|4.541666666666667|   12|
|   52767|              4.5|    5|
|    7075|              4.5|    5|
|     858|           4.4875|  200|
|     318|4.487138263665595|  311|
|    1948|4.458333333333333|   12|
|    8132|4.454545454545454|   11|
|    1945|4.448275862068965|   29|
|    3167|           4.4375|    8|
|    3310|           4.4375|    8|
|     926|4.434210526315789|   38|
|     969|             4.42|   50|
|   77658|4.416666666666667|    6|
|    3035|4.411764705882353|   17|
+--------+-----------------+-----+
only showing top 20 rows



In [12]:
final = users_movies.join(movies, users_movies.movie_ID==movies.movie_ID).select(users_movies.movie_ID, movies.movie_name, users_movies.AVG).orderBy(users_movies.AVG.desc())
final.show(5)

+--------+--------------------+-----------------+
|movie_ID|          movie_name|              AVG|
+--------+--------------------+-----------------+
|     116|Anne Frank Rememb...|             4.75|
|     178|Love & Human Rema...|              4.7|
|    7116|Diabolique (Les d...|              4.7|
|    1939|"Best Years of Ou...|4.636363636363637|
|    5114|"Bad and the Beau...|              4.6|
+--------+--------------------+-----------------+
only showing top 5 rows

